# **[GPT4free Jupyter Notebook (leave a star)](https://github.com/DanielShemesh/gpt4free-colab/)**

This program is licensed under the GNU GPL v3.

&copy; 2023 Made by [Daniel Shemesh](https://github.com/DanielShemesh).
  
This Jupyter Notebook contains code written by [Daniel Shemesh](https://github.com/DanielShemesh). It is based on work from the GitHub repository by xtekky ([GPT4free](https://github.com/xtekky/gpt4free/)). The code in this notebook is released under the terms of the GNU General Public License v3, which allows for free redistribution and modification of the code in accordance with the terms of the license.

## Disclaimer

This repository uses third-party APIs and AI models and is not associated with or endorsed by the API providers or the original developers of the models. This project is intended for educational purposes only.

Please note the following:

**Disclaimer**: The APIs, services, and trademarks mentioned in this repository belong to their respective owners. This project is not claiming any right over them.

**Responsibility**: The author of this repository is not responsible for any consequences arising from the use or misuse of this repository or the content provided by the third-party APIs and any damage or losses caused by users' actions.

**Educational Purposes Only**: This repository and its content are provided strictly for educational purposes. By using the information and code provided, users acknowledge that they are using the APIs and models at their own risk and agree to comply with any applicable laws and regulations.


In [ ]:
#@markdown #Install requirements

%%capture captured_output

!git clone https://github.com/xtekky/gpt4free
%cd gpt4free
!mv testing/* ./
!pip install -r requirements.txt
!pip install ipywidgets

#Instructions
##Poe/Quora
* Sign up to [Poe](https://www.poe.com) and head over to the site
* ctrl+shift+i to open developer console
* Go to Application -> Cookies -> https://poe.com
* Find the **p-b** cookie and copy its value, this will be your **Poe-token**

In [ ]:
#@markdown #Define secrets

import os
import ipywidgets as widgets
from IPython.display import display

def create_input_widget(word):
    widget = widgets.Password(
        value='',
        placeholder=f'Enter your {word}',
        description=f'{word.capitalize()}:',
        disabled=False,
        style={'description_width': 'initial'}
    )
    return widget

def create_save_button(word, widget):
    button = widgets.Button(
        description=f'Save {word.capitalize()}',
        disabled=False,
        button_style='success',
        tooltip=f'Click to save the entered {word} to an environment variable',
        icon='check'
    )
    button.on_click(lambda b: save_word(word, widget))
    return button

def save_word(word, widget):
    os.environ[f'SECRET_{word.upper()}'] = widget.value
    print(f"{word.capitalize()} saved successfully!")

def create_input_and_save_button(word):
    widget = create_input_widget(word)
    button = create_save_button(word, widget)
    display(widget)
    display(button)

create_input_and_save_button('poe-token')

In [ ]:
from markdown_it import MarkdownIt
from IPython.core.display import HTML

import quora
import os
from time import sleep

token = os.environ['SECRET_POE-TOKEN']

#@markdown Choose a model and type your prompt
model = "gpt-3.5-turbo"  # @param ['gpt-3.5-turbo', 'gpt-4', 'sage', 'claude-v1.2', 'claude-instant-v1.0']
prompt = "write a java function that prints the nth fibonacci number. provide example usage" #@param {type: "string"}
#@markdown Checking this will result in a live response
streaming_response = False #@param {type: "boolean"}
#@markdown Render Markdown in normal response (will not work in streaming response)
render_markdown = True #@param {type: "boolean"}
#@markdown Enable Chat Mode
chat_mode = False #@param {type: "boolean"}

def generate_completion(prompt):
    response = quora.Completion.create(model=model, prompt=prompt, token=token)
    return response.completion.choices[0].text

if streaming_response and chat_mode:
    while True:
        user_input = input("You: ")
        response = quora.StreamingCompletion.create(custom_model=model, prompt=user_input, token=token)
        text = response.completion.choices[0].text

        if render_markdown:
            md = MarkdownIt()
            html = md.render(text)
            display(HTML(html))
        else:
            print(text)
elif streaming_response and not chat_mode:
    for response in quora.StreamingCompletion.create(custom_model=model, prompt=prompt, token=token):
        text = response.completion.choices[0].text
        print(text)
elif chat_mode:
    while True:
        user_input = input("You: ")
        text = generate_completion(user_input)

        if render_markdown:
            md = MarkdownIt()
            html = md.render(text)
            display(HTML(html))
        else:
            print(text)
else:
    text = generate_completion(prompt)

    if render_markdown:
        md = MarkdownIt()
        html = md.render(text)
        display(HTML(html))
    else:
        print(text)